In [4]:
import pyaudio
import wave
import whisper
import time
import warnings
warnings.filterwarnings('ignore')
from transformers import pipeline
import threading

In [9]:
class System:
    def __init__(self):
        self.audio = pyaudio.PyAudio()
        self.stream = self.audio.open(format=pyaudio.paInt16, channels=1, rate=44100, input=True, frames_per_buffer=1024)
        self.model = pipeline(model="seifooo/names-whisper-en")  # change to "your-username/the-name-you-picked"
        self.duration = 1
        self.filename = 'audio.wav'
        self.text = ""
        self.result_arr = []
        self.frames = []
        
    def transcribe(self, audio):
        text = self.model(audio)
        return text
        
    def record_audio(self):
        start_time = time.time()
        
        while (time.time() - start_time) < self.duration:
            data = self.stream.read(1024)
            self.frames.append(data)

        with wave.open(self.filename, 'wb') as wf:
            wf.setnchannels(1)
            wf.setsampwidth(self.audio.get_sample_size(pyaudio.paInt16))
            wf.setframerate(44100)
            wf.writeframes(b''.join(self.frames))
            
    def transcribe_audio(self):
        result = self.transcribe(self.filename)
        self.result_arr.append(result["text"])
        if "." in result["text"]:
            self.duration = 1
        else:
            self.duration+=1
        #print("duration {}".format(self.duration))
    
    def check_result(self):
        self.text = self.result_arr[len(self.result_arr)-1]
        # check unconfirmed and confirmed words based on frequancy in the result_arr
        # example: hi we are going to spain
        #          hi where going to spain today is a good day
        # where, today, is, a, good, day are unconfirmed but the rest is confirmed
        # next iteration: hi we are going to spain today is a good day and we like pizza
        # we are gets confirmed since it is repeated twice and so is today is a good day
        print(self.text, end="\r")
            
    def record_and_transcribe(self):
        try:
            print("Starting Record")
            while True:                
                self.record_audio()
                self.transcribe_audio()
                self.check_result()
        
        except KeyboardInterrupt:
            pass

        self.stream.stop_stream()
        self.stream.close()
        self.audio.terminate()



In [ ]:
system = System()
system.record_and_transcribe()

Starting Record
